# Geodata at block level
## including: crime data, census data, etc

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point
from shapely import wkt
from shapely.geometry import *
import json

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline

## crime data

In [2]:
crime_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/crime/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv'
df_crime = pd.read_csv(crime_file)

recent_crime_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/crime/Police_Department_Incident_Reports__2018_to_Present.csv'
df_recent_crime = pd.read_csv(recent_crime_file)

In [3]:
df_crime.columns

Index(['PdId', 'IncidntNum', 'Incident Code', 'Category', 'Descript',
       'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X',
       'Y', 'location', 'SF Find Neighborhoods 2 2',
       'Current Police Districts 2 2', 'Current Supervisor Districts 2 2',
       'Analysis Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2',
       'DELETE - Police Districts 2 2', 'DELETE - Supervisor Districts 2 2',
       'DELETE - Zip Codes 2 2', 'DELETE - Neighborhoods 2 2',
       'DELETE - 2017 Fix It Zones 2 2',
       'Civic Center Harm Reduction Project Boundary 2 2',
       'Fix It Zones as of 2017-11-06  2 2', 'DELETE - HSOC Zones 2 2',
       'Fix It Zones as of 2018-02-07 2 2',
       'CBD, BID and GBD Boundaries as of 2017 2 2',
       'Areas of Vulnerability, 2016 2 2',
       'Central Market/Tenderloin Boundary 2 2',
       'Central Market/Tenderloin Boundary Polygon - Updated 2 2',
       'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2',
       

In [4]:
df_recent_crime.columns

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude',
       'point', 'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'HSOC Zones as of 2018-06-05', 'OWED Public Spaces',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Parks Alliance CPSI (27+TL sites)', 'ESNCAG - Boundary File',
       'Areas of Vulnerability, 2016'],
      dtype='object')

In [5]:
df_crime['location'][0]

'POINT (-122.41982062806301 37.782008780606)'

In [6]:
df_recent_crime['point']

0                                              NaN
1                                              NaN
2                                              NaN
3         (37.75482657770952, -122.39772873392515)
4         (37.76653957529556, -122.42204381448558)
                            ...                   
385982    (37.78880754257507, -122.41188565874671)
385983    (37.79226261825945, -122.43620350896612)
385984    (37.78829319282882, -122.40840155198468)
385985    (37.74123427141981, -122.47449422499847)
385986    (37.78829319282882, -122.40840155198468)
Name: point, Length: 385987, dtype: object

In [9]:
df_crime['Date'][0]

'11/27/2007'

In [11]:
df_crime['Date'] = pd.to_datetime(df_crime['Date'])
df_crime['year'] = df_crime['Date'].dt.year

In [12]:
df_crime['year'].unique()

array([2007, 2004, 2003, 2013, 2011, 2012, 2005, 2008, 2006, 2010, 2009,
       2015, 2016, 2014, 2017, 2018], dtype=int64)

In [28]:
df_crime['year'].value_counts()

2015    153771
2017    151759
2013    149639
2016    148379
2014    147056
2003    145073
2004    144323
2005    138807
2012    137334
2008    137113
2009    136218
2006    133726
2007    133601
2010    129750
2011    128563
2018     45841
Name: year, dtype: int64

In [29]:
df_recent_crime['Incident Year'].value_counts()

2018    153789
2019    149163
2020     83035
Name: Incident Year, dtype: int64

In [15]:
df_crime['Category'].unique()

array(['OTHER OFFENSES', 'NON-CRIMINAL', 'SUSPICIOUS OCC',
       'LARCENY/THEFT', 'DRUG/NARCOTIC', 'ASSAULT', 'PROSTITUTION',
       'WARRANTS', 'RECOVERED VEHICLE', 'BURGLARY', 'VEHICLE THEFT',
       'ROBBERY', 'SEX OFFENSES, FORCIBLE', 'MISSING PERSON', 'VANDALISM',
       'WEAPON LAWS', 'TRESPASS', 'SECONDARY CODES', 'FRAUD',
       'FORGERY/COUNTERFEITING', 'STOLEN PROPERTY', 'KIDNAPPING',
       'DISORDERLY CONDUCT', 'FAMILY OFFENSES', 'SUICIDE', 'DRUNKENNESS',
       'EMBEZZLEMENT', 'EXTORTION', 'DRIVING UNDER THE INFLUENCE',
       'ARSON', 'LIQUOR LAWS', 'SEX OFFENSES, NON FORCIBLE', 'LOITERING',
       'PORNOGRAPHY/OBSCENE MAT', 'BAD CHECKS', 'BRIBERY', 'GAMBLING',
       'TREA'], dtype=object)

In [17]:
df_recent_crime['Incident Category'].unique()

array(['Recovered Vehicle', 'Larceny Theft', 'Lost Property', 'Assault',
       'Malicious Mischief', 'Non-Criminal', 'Weapons Offense',
       'Missing Person', 'Other', 'Burglary',
       'Offences Against The Family And Children',
       'Miscellaneous Investigation', 'Other Miscellaneous',
       'Disorderly Conduct', 'Suspicious Occ', 'Other Offenses',
       'Robbery', 'Motor Vehicle Theft', 'Family Offense', 'Arson',
       'Case Closure', 'Suicide', 'Fraud', 'Traffic Violation Arrest',
       'Stolen Property', 'Drug Offense', 'Vehicle Misplaced',
       'Fire Report', 'Warrant', 'Forgery And Counterfeiting',
       'Courtesy Report', 'Sex Offense', 'Traffic Collision', 'Vandalism',
       'Weapons Carrying Etc', 'Embezzlement', nan, 'Vehicle Impounded',
       'Rape', 'Human Trafficking (A), Commercial Sex Acts',
       'Drug Violation', 'Motor Vehicle Theft?', 'Homicide', 'Gambling',
       'Prostitution', 'Civil Sidewalks', 'Liquor Laws', 'Suspicious',
       'Weapons Offenc

In [30]:
# makes it easy for crime data: only includes: year, category, location
df_crime['type'] = 0
df_crime.loc[(df_crime['Category'] == 'ARSON'),'type'] = 1

df_recent_crime['type'] = 0
df_recent_crime.loc[(df_recent_crime['Incident Category'] == 'Arson'),'type'] = 1

In [47]:
df_crime['X']

0         -122.419821
1         -122.471221
2         -122.475288
3         -122.423538
4         -122.403631
              ...    
2160948   -122.410769
2160949   -122.394866
2160950   -122.457558
2160951   -122.502220
2160952   -122.393946
Name: X, Length: 2115112, dtype: float64

In [48]:
# makes them geodataframe

# df_crime: get rid of 2018, and create geometry using 'location'
df_crime = df_crime[df_crime['year']<2018]
gdf_crime = gpd.GeoDataFrame(
    df_crime, geometry=gpd.points_from_xy(df_crime.X, df_crime.Y))




In [36]:

# df_recent_crime: using 'point', latiture, longitude

#df_recent_crime.dropna(axis='point')
# 'Latitude', 'Longitude',

gdf_recent_crime = gpd.GeoDataFrame(
    df_recent_crime, geometry=gpd.points_from_xy(df_recent_crime.Longitude, df_recent_crime.Latitude))

In [49]:
gdf_crime.head()

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,...,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2,year,type,geometry
0,7121491514040,71214915,14040,OTHER OFFENSES,INDECENT EXPOSURE,Tuesday,2007-11-27,13:53,NORTHERN,"ARREST, CITED",...,8.0,2.0,1.0,1.0,1.0,NaN,21.0,2007,0,POINT (-122.41982 37.78201)
1,7036663851040,70366638,51040,NON-CRIMINAL,AIDED CASE,Tuesday,2007-03-20,23:10,TARAVAL,NONE,...,NaN,1.0,NaN,NaN,NaN,NaN,46.0,2007,0,POINT (-122.47122 37.74230)
2,4059322571000,40593225,71000,NON-CRIMINAL,LOST PROPERTY,Thursday,2004-05-20,12:00,TARAVAL,NONE,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2004,0,POINT (-122.47529 37.73745)
3,3085157264070,30851572,64070,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Thursday,2003-07-17,17:35,INGLESIDE,PROSECUTED BY OUTSIDE AGENCY,...,NaN,1.0,NaN,NaN,NaN,NaN,84.0,2003,0,POINT (-122.42354 37.74383)
4,13067727606304,130677276,6304,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Thursday,2013-08-15,04:42,SOUTHERN,NONE,...,NaN,1.0,NaN,NaN,NaN,NaN,33.0,2013,0,POINT (-122.40363 37.76666)


In [43]:
gdf_crime.columns

Index(['PdId', 'IncidntNum', 'Incident Code', 'Category', 'Descript',
       'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X',
       'Y', 'location', 'SF Find Neighborhoods 2 2',
       'Current Police Districts 2 2', 'Current Supervisor Districts 2 2',
       'Analysis Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2',
       'DELETE - Police Districts 2 2', 'DELETE - Supervisor Districts 2 2',
       'DELETE - Zip Codes 2 2', 'DELETE - Neighborhoods 2 2',
       'DELETE - 2017 Fix It Zones 2 2',
       'Civic Center Harm Reduction Project Boundary 2 2',
       'Fix It Zones as of 2017-11-06  2 2', 'DELETE - HSOC Zones 2 2',
       'Fix It Zones as of 2018-02-07 2 2',
       'CBD, BID and GBD Boundaries as of 2017 2 2',
       'Areas of Vulnerability, 2016 2 2',
       'Central Market/Tenderloin Boundary 2 2',
       'Central Market/Tenderloin Boundary Polygon - Updated 2 2',
       'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2',
       

In [44]:
gdf_recent_crime.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Current Supervisor Districts,Analysis Neighborhoods,HSOC Zones as of 2018-06-05,OWED Public Spaces,Central Market/Tenderloin Boundary Polygon - Updated,Parks Alliance CPSI (27+TL sites),ESNCAG - Boundary File,"Areas of Vulnerability, 2016",type,geometry
0,2020/08/15 08:56:00 AM,2020/08/15,08:56,2020,Saturday,2020/08/15 08:56:00 AM,95300907041,953009,200474239,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,POINT (nan nan)
1,2020/08/15 09:40:00 AM,2020/08/15,09:40,2020,Saturday,2020/08/15 06:21:00 PM,95322706244,953227,206121692,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,POINT (nan nan)
2,2018/02/24 10:00:00 PM,2018/02/24,22:00,2018,Saturday,2018/03/02 10:13:00 AM,64174871000,641748,186051531,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,POINT (nan nan)
3,2020/08/16 03:13:00 AM,2020/08/16,03:13,2020,Sunday,2020/08/16 03:14:00 AM,95319604083,953196,200491669,202290313.0,...,9.0,26.0,NaN,NaN,NaN,NaN,NaN,2.0,0,POINT (-122.39773 37.75483)
4,2020/08/16 03:38:00 AM,2020/08/16,03:38,2020,Sunday,2020/08/16 04:56:00 AM,95326228100,953262,200491738,202290404.0,...,2.0,20.0,3.0,NaN,NaN,NaN,NaN,2.0,0,POINT (-122.42204 37.76654)


In [52]:
gdf_crime = gdf_crime[['year','type','geometry']]

gdf_recent_crime = gdf_recent_crime[['Incident Year','type','geometry']]

gdf_recent_crime= gdf_recent_crime.rename(columns={"Incident Year": "year"})

In [54]:
gdf_crime_all = pd.concat([gdf_crime,gdf_recent_crime])

In [56]:
type(gdf_crime_all)

geopandas.geodataframe.GeoDataFrame

## census block

In [7]:
block_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/Census 2010_ Blocks for San Francisco.geojson'
df_block = gpd.read_file(block_file)

In [8]:
df_block.columns

Index(['statefp10', 'mtfcc10', 'uace10', 'name10', 'intptlat10', 'blockce10',
       'awater10', 'funcstat10', 'aland10', 'geoid10', 'tractce10',
       'intptlon10', 'ur10', 'countyfp10', 'geometry'],
      dtype='object')

In [21]:
df_block['geoid10'][0]

'060750213002002'

In [22]:
df_block['blockce10'][0]

'2002'

### join block with crime

In [57]:
# make sure the two dataset have the same projection
gdf_crime_all.crs = df_block.crs

In [63]:
# select the types of inspection before inputing this function
# these are both GDF! need to convert the crime data into GDF first
def process_crime(gdf_block, gdf_crime, year):
    
    # in order to spatial join, only need id and geometry to keep it simple
    gdf_block_simple = gdf_block[['geoid10', 'geometry']]
    
    gdf_crime = gdf_crime[(gdf_crime['year'] == year)]

    # make sure the two dataset have the same projection
    gdf_crime_all.crs = gdf_block_simple.crs
    
    # separate arson and other crimes
    gdf_arson = gdf_crime[gdf_crime['type']==1]
    gdf_other = gdf_crime[gdf_crime['type']==0]
    
    #join arson
    arson_sjoin = gpd.sjoin(gdf_arson, gdf_block_simple, how="inner", op='within')
    #join other crime
    other_sjoin = gpd.sjoin(gdf_other, gdf_block_simple, how="inner", op='within')
    
    # dropnas
    arson_sjoin = arson_sjoin.dropna()
    other_sjoin = other_sjoin.dropna()
    
    
    # group by parcel id
    grouped = arson_sjoin.groupby('geoid10').size()
    df_arson_block = grouped.to_frame().reset_index()
    df_arson_block.columns = ['geoid10','arson_count']
    
    # group by parcel id
    grouped = other_sjoin.groupby('geoid10').size()
    df_other_block = grouped.to_frame().reset_index()
    df_other_block.columns = ['geoid10','other_crime_count']
    
    # merge
    df_crime_block = df_arson_block.merge(df_other_block,on='geoid10', how='outer')
    
    df_crime_block['arson_count'] = df_crime_block['arson_count'].fillna(0)
    df_crime_block['other_crime_count'] = df_crime_block['other_crime_count'].fillna(0)
    
    df_crime_block['year'] = year 
    
    return df_crime_block

In [67]:
# assemble dataset for crime block, for year 2007-2019
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# initiate df
df_crime_year = pd.DataFrame()

for year in range(2007,2020):
    
    df_crime_year = pd.concat([df_crime_year,process_crime(df_block, gdf_crime_all, year)])
    print(year, end = ', ')

2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 

In [68]:
df_crime_year.shape

(70318, 4)

In [70]:
df_crime_year.head()

,geoid10,arson_count,other_crime_count,year
0,060750101001006,1.0,63.0,2007
1,060750101001019,1.0,16.0,2007
2,060750101001023,1.0,34.0,2007
3,060750101002005,1.0,39.0,2007
4,060750101002011,1.0,44.0,2007


In [74]:
df_crime_year['block_id'] = pd.to_numeric(df_crime_year.geoid10, errors='coerce').astype(np.int64)

In [75]:
df_crime_year.head()

,geoid10,arson_count,other_crime_count,year,block_id
0,060750101001006,1.0,63.0,2007,60750101001006
1,060750101001019,1.0,16.0,2007,60750101001019
2,060750101001023,1.0,34.0,2007,60750101001023
3,060750101002005,1.0,39.0,2007,60750101002005
4,060750101002011,1.0,44.0,2007,60750101002011


## access data and census data

In [18]:
access_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/ba_block_variables.csv'
df_access = pd.read_csv(access_file)

In [19]:
df_access.columns

Index(['block_id', 'x', 'y', 'res_rents', 'res_values', 'square_meters_land',
       'puma10_id', 'residential_unit_capacity', 'employment_capacity',
       'rent_impute',
       ...
       'tracts_prop_persons_8', 'tracts_prop_persons_9', 'nodes_du_1500m',
       'nodes_ave_year_built_3000m', 'tracts_prop_persons_5',
       'tracts_prop_persons_6', 'tracts_prop_persons_7',
       'tracts_prop_persons_1', 'tracts_prop_persons_2',
       'tracts_prop_persons_3'],
      dtype='object', length=1015)

In [25]:
df_access['block_id'][1]

60014271001001

In [26]:
df_access.shape

(109228, 1015)

In [76]:
df_final = df_crime_year.merge(df_access,on='block_id', how='outer')

In [77]:
df_final.head()

,geoid10,arson_count,other_crime_count,year,block_id,x,y,res_rents,res_values,square_meters_land,...,tracts_prop_persons_8,tracts_prop_persons_9,nodes_du_1500m,nodes_ave_year_built_3000m,tracts_prop_persons_5,tracts_prop_persons_6,tracts_prop_persons_7,tracts_prop_persons_1,tracts_prop_persons_2,tracts_prop_persons_3
0,060750101001006,1.0,63.0,2007.0,60750101001006,-122.416532,37.807644,1202.0,759900.0,15459,...,0.000478,0.0,8.426771,1947.471558,0.006699,0.002871,0.000478,0.564115,0.314354,0.078469
1,060750101001006,NaN,50.0,2008.0,60750101001006,-122.416532,37.807644,1202.0,759900.0,15459,...,0.000478,0.0,8.426771,1947.471558,0.006699,0.002871,0.000478,0.564115,0.314354,0.078469
2,060750101001006,NaN,36.0,2009.0,60750101001006,-122.416532,37.807644,1202.0,759900.0,15459,...,0.000478,0.0,8.426771,1947.471558,0.006699,0.002871,0.000478,0.564115,0.314354,0.078469
3,060750101001006,NaN,47.0,2010.0,60750101001006,-122.416532,37.807644,1202.0,759900.0,15459,...,0.000478,0.0,8.426771,1947.471558,0.006699,0.002871,0.000478,0.564115,0.314354,0.078469
4,060750101001006,NaN,45.0,2011.0,60750101001006,-122.416532,37.807644,1202.0,759900.0,15459,...,0.000478,0.0,8.426771,1947.471558,0.006699,0.002871,0.000478,0.564115,0.314354,0.078469


In [82]:
# the data is too large: select useful data to output as csv
cols_to_use = ['block_id','arson_count','other_crime_count','year',
        'nodes_population_1500m','block_groups_total_jobs', 'block_groups_median_children', 
       'block_groups_median_income', 'prop_tenure_1', 'nodes_low_income_hh_1500m', 'nodes_high_income_hh_1500m', 
       'nodes_jobs_3000m','nodes_jobs_20km', 'nodes_population_400m', 'nodes_population_800m', 
       'block_groups_prop_race_of_head_1','block_groups_prop_race_of_head_2', 'block_groups_prop_race_of_head_3', 
       'block_groups_prop_race_of_head_7','block_groups_prop_race_of_head_8','block_groups_prop_race_of_head_6',
       'pumas_density_residential_units','block_groups_density_jobs', 
       'nodes_jobs_1500m_4445','nodes_jobs_3000m_4445', 
       'nodes_du_5000m','nodes_du_800m','block_groups_median_rent',
       'block_groups_median_persons', 'block_groups_median_age_of_head', 'nodes_ave_year_built_800m']

df_output = df_final[cols_to_use]

In [84]:
df_output.to_csv('sf_crime_census.csv', index=False)